# Intersection Density

Generating measures of intersection density to a hex grid using PostGIS and some spatial python libraries

In [1]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import osmnx as ox
import seaborn as sns
import matplotlib.pyplot as plt

The postgis database has a table named `hex_grid_200m` and 

In [2]:
dfhex = gpd.read_file("../input_data/hex_grid/hex_grid_200m_test_subset.shp").to_crs(epsg=32617)
dfhex = dfhex.set_index('id')

In [3]:
# get boundary for the OSM
boundary = dfhex.geometry.to_crs(epsg = 4326).unary_union

KeyboardInterrupt: 

Selecting and downloading the OSM network graph using custom filters for selecting ways

In [ ]:
filter_drive = ('["area"!~"yes"]'
            '["highway"!~"motor|trunk|foot|proposed|construction|abandoned|platform|raceway|foot|cycleway"]'
            '["service"!~"driveway|emergency_access|parking_aisle|drive-through"]'
            '["access"!~"private"]')
filter_walk = ('["area"!~"yes"]'
            '["highway"!~"motor|trunk|proposed|construction|abandoned|platform|raceway"]'
            '["foot"!~"no"]'
            '["footway"!~"sidewalk"]'
            '["service"!~"private|parking_aisle"]'
            '["access"!~"private"]')
G = ox.graph_from_polygon(boundary, custom_filter=filter_drive)
G = ox.project_graph(G)
# fig, ax = ox.plot_graph(G, node_size=30, node_color='#66cc66') # for plotting :)

Grabbing the street nodes from the graph into a geodataframe

In [ ]:
nodes_all = ox.graph_to_gdfs(G, edges=False)
nodes_all = nodes_all.to_crs(epsg=32617)

pulling out the number of streets that connect to each edge and joining to the nodes dataset

In [ ]:
streets_per_node = ox.count_streets_per_node(G)
streets_per_node = pd.DataFrame.from_dict(data=streets_per_node,orient='index')
streets_per_node.columns = ["streets_n"]
nodes_all = nodes_all.join(streets_per_node)

Spatial join of the hex grid to the nodes

In [ ]:
nodes_per_hex = gpd.sjoin(nodes_all, dfhex, how="inner", op='intersects')
del nodes_per_hex['highway'],nodes_per_hex['x'],nodes_per_hex['y'],nodes_per_hex['lat'],nodes_per_hex['lon'],nodes_per_hex['geometry']

Subsetting to only get those with 3 or more and 4 or more street connections

In [ ]:
nodes_per_hex_4up = nodes_per_hex.query('streets_n >= 4')
nodes_per_hex_3up = nodes_per_hex.query('streets_n >= 3')

Group by hex summing the number of nodes

In [ ]:
nodes_per_hex_4up = nodes_per_hex_4up.groupby('index_right')['streets_n'].count()
nodes_per_hex_3up = nodes_per_hex_3up.groupby('index_right')['streets_n'].count()

Join outter to the hex bounds

In [ ]:
nodes_per_hex_3up = (pd.concat((dfhex,nodes_per_hex_3up), axis=1, join='outer'))
del nodes_per_hex_3up["geometry"]
nodes_per_hex_3up.fillna(0, inplace=True)
nodes_per_hex_3up.columns = ["nodes_road_3up"]

nodes_per_hex_4up = (pd.concat((dfhex,nodes_per_hex_4up), axis=1, join='outer'))
del nodes_per_hex_4up["geometry"]
nodes_per_hex_4up.fillna(0, inplace=True)
nodes_per_hex_4up.columns = ["nodes_road_4up"]

Join the data to one dataframe and output to a csv

In [ ]:
hex_int_nodes = nodes_per_hex_3up.join(nodes_per_hex_4up)
hex_int_nodes.index.names = ['hexid']
hex_int_nodes.to_csv("gen_code/hexdata_intersections_road.csv", encoding='utf-8', index=True)